# Project 2 - WRITEUP

## 1. DH PARAMETERS TABLE

In [1]:
from sympy import symbols, cos, sin, pi, simplify, sqrt, atan2
from sympy.matrices import Matrix
import numpy as np
from numpy import array

## Create symbols for joint variables
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') # theta_i
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')


### KUKA KR210 ###
s = {
    alpha0: 0,     a0:    0,          d1: 0.75,
    
    alpha1: -pi/2, a1:    0.35,       d2: 0,       q2: (q2 - pi/2),
    
    alpha2: 0,     a2:    1.25,       d3: 0,
    
    alpha3: -pi/2, a3:  -0.054,       d4: 1.50,
    
    alpha4: pi/2,  a4:       0,       d5: 0,
    
    alpha5: -pi/2, a5:       0,       d6: 0,
    
    alpha6: 0,     a6:       0,       d7: 0.303,    q7:0
    
}


Based on URDF file and Project 2 video lectures, we can come up with the above DH table. Also, we apply the right-hand rule to determine direction of all the coordinates. 


Because the joint locations are not exactly the same as the chosen origin of the frame coordinate, the diagram below is based purely on the DH table above information and assume that the joint and origin of the frame will be the same.

[INSERT DRAWING DIAGRAM HERE]

## 2 . TRANSFORMATION MATRICES